In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
from functools import *

dataset = pd.read_csv('mnist/mnist_train.csv')
testset = pd.read_csv('mnist/mnist_test.csv')

In [64]:
class Perceptron(object):
    def __init__(self,input_size, number=-1, lr=1, epochs=100):
        self.W = np.zeros(input_size+1)
        self.epochs = epochs
        self.lr = lr
        self.number = number
    
    def activation_fn(self, x):
        return (1 / (1 + np.exp(-x)))
 
    def predict(self, x):
        z = self.W.T.dot(x)
        a = self.activation_fn(z)
        return a
    
    def guess_new(self, x):
        z = self.W[1:].dot(x) + self.W[0]
        return self.activation_fn(z)
 
    def fit(self, X, d):
        for _ in range(self.epochs):
            for i in range(d.shape[0]):
                x = np.insert(X[i], 0, 1)
                y = self.predict(x)
                e = d[i] - y
                self.W = self.W + self.lr * e * x

In [72]:
train = np.array(dataset)[0:300,:]
test = np.array(testset)[0:300,:]

In [73]:
#Train perceptron by correspondent number

perseptrons = [Perceptron(input_size=train.shape[1]-1, lr=0.001, epochs=100,number=i) for i in range(10)]

for perceptron in perseptrons:
    perceptron.fit(train[:, 1:], train[0:, 0])
    #Get Accuracy
    total = 200
    err_total = 0
    acc_total = 0
    for test_case in test:
        prediction = round(perceptron.guess_new(test_case[1:]))
        if (prediction == 1 and perceptron.number == test_case[0]):
            acc_total +=1
        else:
            err_total +=1
    print('Accuracy of Perceptron number ' + str(perceptron.number) + ' is: ' + str(float(acc_total)/float(total)))

Accuracy of Perceptron number 0 is: 0.12
Accuracy of Perceptron number 1 is: 0.205
Accuracy of Perceptron number 2 is: 0.16
Accuracy of Perceptron number 3 is: 0.12
Accuracy of Perceptron number 4 is: 0.19
Accuracy of Perceptron number 5 is: 0.145
Accuracy of Perceptron number 6 is: 0.12
Accuracy of Perceptron number 7 is: 0.165
Accuracy of Perceptron number 8 is: 0.105
Accuracy of Perceptron number 9 is: 0.17
